In [4]:
from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
import operator

load_dotenv()

True

In [5]:
from typing import Literal


class QuadraticState(TypedDict):
    a: int
    b: int
    c: int
    discriminant: float
    equation: str
    result: str

def calculate_discriminant(state: QuadraticState) -> dict:
    a = state['a']
    b = state['b']
    c = state['c']
    discriminant = b**2 - 4*a*c
    return {'discriminant': discriminant}

def real_roots(state: QuadraticState) -> dict:
    a = state['a']
    b = state['b']
    discriminant = state['discriminant']
    root1 = (-b + discriminant**0.5) / (2*a)
    root2 = (-b - discriminant**0.5) / (2*a)
    return {'result': f"The roots are real and different: {root1} and {root2}"}

def equal_roots(state: QuadraticState) -> dict:
    a = state['a']
    b = state['b']
    root = -b / (2*a)
    return {'result': f"The roots are real and equal: {root}"}

def no_real_roots(state: QuadraticState) -> dict:
    return {'result': "The roots are complex and imaginary."}

def check_condition(state: QuadraticState) -> Literal["real_roots", "equal_roots", "no_real_roots"]:
    discriminant = state['discriminant']
    if discriminant > 0:
        return "real_roots"
    elif discriminant == 0:
        return "equal_roots"
    else:
        return "no_real_roots"

In [6]:
graph = StateGraph(QuadraticState)

graph.add_node("calculate_discriminant", calculate_discriminant)
graph.add_node("real_roots", real_roots)
graph.add_node("equal_roots", equal_roots)
graph.add_node("no_real_roots", no_real_roots)

graph.add_edge(START, "calculate_discriminant")
graph.add_conditional_edges("calculate_discriminant", check_condition)
graph.add_edge("real_roots", END)
graph.add_edge("equal_roots", END)
graph.add_edge("no_real_roots", END)

compiled_graph = graph.compile()
result = compiled_graph.invoke({"a": 1, "b": -3, "c": 2})
print(result['result'])

The roots are real and different: 2.0 and 1.0
